# the program is used to cut eps data 
# Input
# mdir: direction stored raw data, the dir include some station sub dir 
# mdir structure
# |-st1
# | |-DATALST
# | |-LOGFILES
# | |-MiniSeed
# |-st2
# outdir: cut Date output dir
# outdir structure
#   |-year
#   |  |-yearmonth
#   |       |-deepestfile
# starttime: the record starttime
# seg_len: cut data segment length
# shift: the shift time between the near segments
# Output:
# segment.sac: stored in outdir
# gaps.txt: the gap of series

In [1]:
import os 
import glob
import warnings
import obspy
from obspy import UTCDateTime

In [2]:
warnings.filterwarnings('ignore')

In [12]:
mdir = r"F:\YB_backup"
outdir=r"E:\YB\station"
starttime = UTCDateTime("20210626")
seg_len = 86400  # second
shift = 86400 

In [4]:
st_lst=glob.glob(mdir+"/YB10_jiami/")

In [6]:
# The function is used to read DATALST FILE
# intput: 
# filename is DATALST FILE absolute path
# return:
# Date file name list
def rd_dl(filename):
    with open(filename, 'r') as f:
        lines = f.readlines()
    #print(lines[::-1])
    # stored date file name
    f_lst = []
    for line in lines[::-1]:
        fnm = line[32:44]
        f_lst.append(fnm)
        # only read the data recorded at the last deployment
        if line[:4] =="0001":
            break
    f_lst.reverse()
    return f_lst

In [7]:
# change head virabel
# Input:
# s1: cut data stream
# Output:
# s1: changed header stream
def ch_hd(s1):
    for tr in s1:
        time = tr.stats.starttime
        tr.stats['sac']={}
        tr.stats.sac['nzyear']=time.year
        tr.stats.sac['nzjday']=time.julday
        tr.stats.sac['nzhour']=time.hour
        tr.stats.sac['nzmin']=time.minute
        tr.stats.sac['nzsec']=time.second
        tr.stats.sac['nzmsec']=time.microsecond
        tr.stats.sac['b']=0
    return s1


In [8]:
# write sac files
# Input:
# s1: cutdata stream
# outdir: output dir
def wt_sac(s1, outdir):
    #time = max([trs[i].stats.starttime for i in range(len(trs))])
    s1 = ch_hd(s1)
    for tr in s1:
        year = tr.stats.starttime.year
        month = "{:02d}".format(tr.stats.starttime.month)
        day = "{:02d}".format(tr.stats.starttime.day)
        hour = "{:02d}".format(tr.stats.starttime.hour)
        minute = "{:02d}".format(tr.stats.starttime.minute)
        second = "{:02d}".format(tr.stats.starttime.second)
        subpath = str(year)+"/"+str(year)+month+"/"+str(year)+month+day+hour+minute+second
        ab_path = outdir+"/"+subpath
        try:
            os.makedirs(ab_path)
        except:
            print("the dir has exited")
        trnm=tr.stats.network+"_"+tr.stats.station+"_"+tr.stats.channel+".SAC"
        tr.write(ab_path+"/"+trnm,format="SAC")
        
    
    
    

In [9]:
# write gaps
def wt_gaps(gaps,outdir=outdir):
    fg = open(outdir+"/"+'gaps.txt','w')
    for gap in gaps:
        print("%s\t%s\t%s\t%s\t%s\t%s\t" % (gap[0], gap[1], gap[3], gap[4], gap[5], gap[7]))
    fg.close() 

In [10]:
# cut data
# Input:
# filelst: miniseed data file name list(time order)
# dirnm: dad dir path
# seg_len: segment length (second)
# shift: shift time (second)
# fill_value: fill gap by the value
# outdir: output dir
def cut_data(filelst, dirnm, starttime, seg_len=86400, shift=86400, fill_value=0, outdir=outdir):
    seg_b = starttime
    trs=obspy.Stream()
    gaps=[]# merge gaps between different series
    
    seg_e = seg_b + seg_len
    for fnm in filelst:
        print(fnm)
        fab = dirnm+fnm
        trs += obspy.read(fab)
        # ss is back-up stream 
        ss = trs.copy()
        # merge traces
        trs.merge()
        # get gaps
        if trs.get_gaps!=[]:
            gaps+=trs.get_gaps()
            trs = ss.copy()
            trs.merge(fill_value=fill_value)
        
        # get the maxium(lastest) starttime and minum(earlist) endtime of traces
        max_b=max([trs[i].stats.starttime for i in range(len(trs))])
        min_e=min([trs[i].stats.endtime for i in range(len(trs))])
        print(seg_b, seg_e)
        print(max_b, min_e)
        # *******************cut series******************
        # case 1:
        #   max_b   seg_b              seg_e      min_e
        #   |--------|--------------------|---------|
        #  |---------|--------------------|---------------|
        # |----------|--------------------|------------|
        # cut series segment and shift seg_b and seg_e
        # remain residual series and enter next loop 
        if max_b<=seg_b and min_e>seg_e:
            # s1 is tempera virabel
            s1 = trs.copy()
            s1.trim(seg_b,seg_e, nearest_sample=False)
            wt_sac(s1, outdir)
            print(s1)
            # residual series
            trs.trim(seg_e)
            seg_b +=shift
            seg_e = seg_b+seg_len
            continue
        # case 2:
        #  max_b           min_e  seg_b          seg_e
        #    |---------------|     |               |
        #  |------------------------|
        # |---------------------|
        #  clear out streams and enter next loop
        elif min_e<seg_b:
            trs=obspy.Stream()
            continue
        # case 3:
        #  seg_b           seg_e max_b          min_e
        #    |               |    |--------------|
        #                   |-------------------------|
        #                       |-------------------|
        # shift seg_b and seg_e until seg_e is large than max_b 
        elif seg_e<=max_b:
            while seg_e<=max_b:
                seg_b+=shift
                seg_e=seg_b+seg_len
            continue
        # case 4:
        #    seg_b      max_b      seg_e       min_e
        #      |          |----------|-----------|
        #   |--------------------------------------|
        #           |--------------------------------|
        #  shift seg_b and seg_e  
        elif seg_b<=max_b and seg_e>max_b and min_e>=seg_e:
            while seg_b<=max_b:
                seg_b+=shift
                seg_e = seg_b+seg_len
            continue
        # case 5:
        #    max_b      seg_b   min_e     seg_e
        #      |----------|-------|         |
        #   |-------------|-------------|   |
        #     |-----------|-----------------|---|
        #  enter next loop
        elif seg_e>min_e and seg_b<=min_e and max_b<seg_b:
            continue
        # case 6:
        #   seg_b   max_b      min_e     seg_e
        #     |       |----------|         |
        #        |---------------------------|
        #   |---------------------------|
        # |-------------------------------------|
        #
        elif seg_b<max_b and seg_e>min_e:
            while seg_b<=max_b:
                seg_b+=shift
                seg_e = seg_b+seg_len
            continue  
    wt_gaps(gaps, outdir=outdir)          



            
            
                

        


In [13]:
# the main program
for st in st_lst:
    print(st)
    tr_nm=glob.glob(st+"DATAFILE.LST")[0]
    f_lst = rd_dl(tr_nm)
    cut_data(f_lst,st,starttime, seg_len=seg_len, shift=shift, outdir=outdir)
    
    

F:\YB_backup/YB10_jiami/
E00016GM.28C


FileNotFoundError: [Errno 2] No such file or directory: 'F:\\YB_backup/YB10_jiami/E00016GM.28C'